In [2]:
!pip install flask flask-ngrok


  Obtaining dependency information for flask-ngrok from https://files.pythonhosted.org/packages/af/6c/f54cb686ad1129e27d125d182f90f52b32f284e6c8df58c1bae54fa1adbc/flask_ngrok-0.0.25-py3-none-any.whl.metadata


In [6]:
import pandas as pd
import glob
import json

# Use a single file path
file_path = "updated_metadata.json"  # <-- update filename

rows = []

# Load and flatten the JSON data
with open(file_path, "r") as fh:
    summary = json.load(fh)

    row = {"run_id": summary.get("run_id")}
    row.update({f"param_{k}": v for k, v in summary.get("params", {}).items()})
    row.update({f"metric_{k}": v for k, v in summary.get("metrics", {}).items()})
    row.update({f"tag_{k}": v for k, v in summary.get("tags", {}).items()})
    rows.append(row)

# Create and display the DataFrame
df = pd.DataFrame(rows)
df.columns.to_list()

['run_id',
 'param_bootstrap',
 'param_ccp_alpha',
 'param_class_weight',
 'param_columns_raw',
 'param_criterion',
 'param_database.description',
 'param_database.id',
 'param_database.name',
 'param_database.owner',
 'param_dataset.authors',
 'param_dataset.doi',
 'param_dataset.published',
 'param_dataset.publisher',
 'param_dataset.title',
 'param_dropped_columns',
 'param_feature_names',
 'param_matplotlib_version',
 'param_max_depth',
 'param_max_features',
 'param_max_leaf_nodes',
 'param_max_samples',
 'param_min_impurity_decrease',
 'param_min_samples_leaf',
 'param_min_samples_split',
 'param_min_weight_fraction_leaf',
 'param_numpy_version',
 'param_n_estimators',
 'param_n_features',
 'param_n_features_final',
 'param_n_jobs',
 'param_n_records',
 'param_n_test_samples',
 'param_n_train_samples',
 'param_oob_score',
 'param_os_platform',
 'param_pandas_version',
 'param_python_version',
 'param_random_state',
 'param_retrieval_time',
 'param_seaborn_version',
 'param_shap_v

In [8]:
!pip install streamlit


  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/eb/17/fc425e1d4d86e31b2aaf0812a2ef2163763a0670d671720c7c36e8679323/streamlit-1.44.1-py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/aa/f3/0b6ced594e51cc95d8c1fc1640d3623770d01e4969d29c0bd09945fafefa/altair-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydeck<1,>=0.8.0b4 from https://files.pythonhosted.org/packages/ab/4c/b888e6cf58bd9db9c93f40d1c6be8283ff49d88919231afe93a6bcf61626/pydeck-0.9.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
    --------------------------------------- 0.2/9.8 MB 4.6 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/9.8 MB 7.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.4/9.8 MB 9.9 MB/s eta 0:00:01
   ------- -------------------------------- 1.9/9.8 MB 11.3 MB/s eta 0:00:01
   -

In [9]:
streamlit run vizualization.py


SyntaxError: invalid syntax (2285487456.py, line 1)